# Language Models: Auto-Complete

In this project i will build an auto-complete system.  Auto-complete system is something you may see every day


In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.data.path.append('.')

<a name='1'></a>
## Part 1: Load and Preprocess Data

In [ ]:
with open("en_US.twitter.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


<a name='1.2'></a>
### Part 1.2 Pre-process the data

In [ ]:
def split_to_sentences(data):
    # Split data by linebreak "\n"
    sentences = data.split('\n')

    # Remove leading and trailing spaces from each sentence
    # Drop sentences if they are empty strings.
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences


In [ ]:
# testing the code
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)

split_to_sentences(x)


I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

<a name='ex-02'></a>
The next step is to tokenize sentences (split a sentence into a list of words).
- Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
- Append each tokenized list of words into a list of tokenized sentences.

In [ ]:
def tokenize_sentences(sentences):
    # Initialize the list of lists of tokenized sentences
    tokenized_sentences = []

    # Go through each sentence
    for sentence in sentences:
        # Convert to lowercase letters
        sentence = sentence.lower()

        # Tokenize into a list of words
        tokenized = nltk.word_tokenize(sentence)

        # Append the list of words to the list of lists
        tokenized_sentences.append(tokenized)

    return tokenized_sentences


In [ ]:
# test your code
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

<a name='ex-03'></a>



Using the two functions that we have just implemented to get the tokenized data.
- split the data into sentences
- tokenize those sentences

In [ ]:
def get_tokenized_data(data):
    # Get the sentences by splitting up the data
    sentences = split_to_sentences(data)

    # Tokenize the sentences to get the list of lists of tokens
    tokenized_sentences = tokenize_sentences(sentences)

    return tokenized_sentences


In [ ]:
# test your function
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

### Split into train and test sets

In [ ]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [ ]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])

print("First test sample")
print(test_data[0])

47961 data are split into 38368 train and 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


<a name='ex-04'></a>
For our project implementation, we'll prioritize words based on their frequency of occurrence in the data. This entails:

Utilizing a double for-loop structure: one loop for iterating through sentences and another nested loop for tokenizing within each sentence.
Establishing a threshold, N, to focus on words that appear at least N times in the data, ensuring we capture the most relevant vocabulary.

In [ ]:
def count_words(tokenized_sentences):
    word_counts = {}

    # Loop through each sentence
    for sentence in tokenized_sentences:
        # Go through each token in the sentence
        for token in sentence:
            # If the token is not in the dictionary yet, set the count to 1
            if token not in word_counts:
                word_counts[token] = 1
            # If the token is already in the dictionary, increment the count by 1
            else:
                word_counts[token] += 1

    return word_counts


In [ ]:
# testing the code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

### Handling 'Out of Vocabulary' words

To effectively handle unknown words during prediction, we'll follow these steps:

Generate a list of the most frequent words in the training set, forming the closed vocabulary.
Identify words outside the closed vocabulary, i.e., those with low occurrence frequencies.
Convert these infrequent words to the special token 'unk', representing unknown words.
By implementing this approach, we ensure that the model encounters and learns to handle words it may not have seen frequently during training, thereby enhancing its ability to predict effectively even with out-of-vocabulary words.

<a name='ex-05'></a>
For our project implementation, we need to create a function that refines a text document based on a specified threshold, 'count_threshold'. This function will:

Retain words with counts greater than or equal to 'count_threshold' in the closed vocabulary.
Replace all other words with the token 'unk', representing unknown words.
This function will help us prepare our training data by ensuring that only the most frequent words, along with the 'unk' token, are included in the document, facilitating effective model training and handling of out-of-vocabulary words during prediction

In [ ]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    # Initialize an empty list to contain the words that appear at least 'count_threshold' times.
    closed_vocab = []

    # Get the word counts of the tokenized sentences
    word_counts = count_words(tokenized_sentences)

    # Check each word and its count
    for word, cnt in word_counts.items():
        # Check if the word's count is at least as great as the minimum count threshold
        if cnt >= count_threshold:
            # Append the word to the list
            closed_vocab.append(word)

    return closed_vocab


In [ ]:
# testing the code
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)

Closed vocabulary:
['.', 'are']


<a name='ex-06'></a>
For our project implementation, we'll devise a function to process a text document, taking into account a specified threshold, 'count_threshold'. This function will:

Define the closed vocabulary as containing words that appear 'count_threshold' times or more.
Replace any words not in the closed vocabulary with the special token "<unk>", denoting them as unknown.
By implementing this approach, we ensure that our training data includes only the most frequently occurring words while efficiently handling out-of-vocabulary instances with the designated '<unk>' token.

In [ ]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    # Initialize an empty list to contain the words that appear at least 'count_threshold' times.
    closed_vocab = []

    # Get the word counts of the tokenized sentences
    word_counts = count_words(tokenized_sentences)

    # Check each word and its count
    for word, cnt in word_counts.items():
        # Check if the word's count is at least as great as the minimum count threshold
        if cnt >= count_threshold:
            # Append the word to the list
            closed_vocab.append(word)

    return closed_vocab


In [ ]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


<a name='ex-07'></a>
In our project implementation, we're prepared to process our data efficiently by integrating the functions we've developed. Here's the plan:

Identify tokens that appear at least 'count_threshold' times in the training data to form the closed vocabulary.
Replace tokens with counts below 'count_threshold' with "<unk>" for both training and test data.
This systematic approach ensures that our training and test data are effectively preprocessed, with infrequent tokens replaced by "<unk>" to facilitate robust model training and prediction handling of unknown words.

In [ ]:
def preprocess_data(train_data, test_data, count_threshold):
    # Get the closed vocabulary using the train data
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)

    # For the train data, replace less common words with "<unk>"
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)

    # For the test data, replace less common words with "<unk>"
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary


In [ ]:
# testing the code
tmp_train = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train,
                                                           tmp_test,
                                                           count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


### Preprocess the train and test data
Run the cell below to complete the preprocessing both for training and test sets.

In [ ]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data,
                                                                        test_data,
                                                                        minimum_freq)

In [ ]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']

First preprocessed test sample:
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']

First 10 vocabulary:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the']

Size of vocabulary: 14821


<a name='2'></a>
## Part 2: Develop n-gram based language models

For our project implementation, the next step is to create a function capable of computing the counts of n-grams for any given value of n.

Here's how we'll proceed:

Prepare each sentence by adding (n-1) starting markers s at the beginning, indicating the start of the sentence.
Append an end token "<e>" to the sentence to signify its completion.
Utilize a dictionary to store the counts of n-grams, with keys represented as tuples of n words (not a list) and values denoting the number of occurrences.
It's crucial to note that we'll use tuples instead of lists as keys in the dictionary. This choice is made because tuples are immutable, ensuring that the keys remain consistent and reliable throughout the counting process.

In [ ]:
def count_n_grams(data, n, start_token='<s>', end_token='<e>'):
    # Initialize dictionary of n-grams and their counts
    n_grams = {}

    # Go through each sentence in the data
    for sentence in data:
        # Prepend start token n times and append <e> once
        sentence = [start_token] * n + sentence + [end_token]

        # Convert list to tuple
        # So that the sequence of words can be used as a key in the dictionary
        sentence = tuple(sentence)

        # Use 'm' to indicate the last index where the end of the n-gram is within the sentence.
        m = len(sentence) if n == 1 else len(sentence) - 1
        for i in range(m):
            # Get the n-gram from i to i+n
            n_gram = sentence[i:i + n]

            # Check if the n-gram is in the dictionary
            if n_gram in n_grams:
                # Increment the count for this n-gram
                n_grams[n_gram] += 1
            else:
                # Initialize this n-gram count to 1
                n_grams[n_gram] = 1

    return n_grams


In [ ]:
# testing  code
# CODE REVIEW COMMENT: Outcome does not match expected outcome
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


<a name='ex-09'></a>
Next, estimate the probability of a word given the prior 'n' words using the n-gram counts.

In our project implementation, we need to define a function that computes the probability estimate using n-gram counts and a constant
𝑘
k. Here's how we'll proceed:

The function will accept two dictionaries as input:
'n_gram_counts', where the keys represent n-grams and the values denote the counts of those n-grams.
'n_plus1_gram_counts', which will be used to find the count for the previous n-gram plus the current word.
This function will be essential for estimating probabilities based on the provided n-gram counts and the smoothing constant
𝑘
k.

In [ ]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    # Convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)

    # Set the denominator
    # If the previous n-gram exists in the dictionary of n-gram counts,
    # get its count. Otherwise, set the count to zero.
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

    # Calculate the denominator using the count of the previous n-gram and apply k-smoothing
    denominator = previous_n_gram_count + k * vocabulary_size

    # Define n plus 1 gram as the previous n-gram plus the current word as a tuple
    n_plus1_gram = previous_n_gram + (word,)

    # Set the count to the count in the dictionary, otherwise 0 if not in the dictionary
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0

    # Define the numerator using the count of the n-gram plus current word and apply smoothing
    numerator = n_plus1_gram_count + k

    # Calculate the probability as the numerator divided by denominator
    probability = numerator / denominator

    return probability


In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


### Estimate probabilities for all words

The function defined below loops over all words in vocabulary to calculate probabilities for all possible words.
- This function is provided for you.

In [ ]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    # Convert list to tuple to use it as a dictionary key
    previous_n_gram = tuple(previous_n_gram)

    # Add <e> and <unk> to the vocabulary
    # <s> is not needed since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)

    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)
        probabilities[word] = probability

    return probabilities


In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'cat': 0.2727272727272727,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'this': 0.09090909090909091,
 'i': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [ ]:
# Additional test
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=1)

{'cat': 0.09090909090909091,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'this': 0.18181818181818182,
 'i': 0.18181818181818182,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

### Count and probability matrices

As we have seen so far, the n-gram counts computed above are sufficient for computing the probabilities of the next word.  
- It can be more intuitive to present them as count or probability matrices.
- The functions defined in the next cells return count or probability matrices.
- This function is provided for you.

In [ ]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    # add <e> <unk> to the vocabulary
    # <s> is omitted since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]

    # obtain unique n-grams
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))

    # mapping from n-gram to row
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))

bigram counts


,cat,a,is,like,dog,this,i,<e>,<unk>
"(is,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(dog,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>,)",0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
"(a,)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(i,)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(like,)",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Show trigram counts
print('\ntrigram counts')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))


trigram counts


,cat,a,is,like,dog,this,i,<e>,<unk>
"(like, a)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(is, like)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, this)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(<s>, i)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(this, dog)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(dog, is)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(<s>, <s>)",0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


The following function calculates the probabilities of each word given the previous n-gram, and stores this in matrix form.
- This function is provided for you.

In [ ]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k=1))

bigram probabilities


,cat,a,is,like,dog,this,i,<e>,<unk>
"(is,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(dog,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.181818,0.181818,0.090909,0.090909
"(a,)",0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(this,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(i,)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(like,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909


In [ ]:
print("trigram probabilities")
trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k=1))

trigram probabilities


,cat,a,is,like,dog,this,i,<e>,<unk>
"(like, a)",0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(a, cat)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(is, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, this)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(<s>, i)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(i, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(this, dog)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(dog, is)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, <s>)",0.090909,0.090909,0.090909,0.090909,0.090909,0.181818,0.181818,0.090909,0.090909


Confirm that you obtain the same results as for the `estimate_probabilities` function that you implemented.

<a name='3'></a>
## Part 3: Perplexity

In this section, we'll evaluate our model on the test set using perplexity scores. Here's the plan:

We'll incorporate back-off strategies when necessary to handle unseen n-grams.
Perplexity serves as a critical metric for evaluating our language model's performance.
Essentially, perplexity measures how well the model predicts the test set. A lower perplexity indicates better performance, meaning the model's predictions align well with the actual test data.

<a name='ex-10'></a>
Compute the perplexity score given an N-gram count matrix and a sentence.

In [ ]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    # Length of previous words
    n = len(list(n_gram_counts.keys())[0])

    # Prepend <s> and append <e>
    sentence = ["<s>"] * n + sentence + ["<e>"]

    # Cast the sentence from a list to a tuple
    sentence = tuple(sentence)

    # Length of sentence (after adding <s> and <e> tokens)
    N = len(sentence)

    # The variable product_pi will hold the product
    # that is calculated inside the n-root
    product_pi = 1.0

    # Index t ranges from n to N - 1, inclusive on both ends
    for t in range(n, N):
        # Get the n-gram preceding the word at position t
        n_gram = sentence[t - n:t]

        # Get the word at position t
        word = sentence[t]

        # Estimate the probability of the word given the n-gram
        # using the n-gram counts, n-plus1-gram counts,
        # vocabulary size, and smoothing constant
        probability = estimate_probability(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)

        # Update the product of the probabilities
        # This 'product_pi' is a cumulative product
        # of the (1/P) factors that are calculated in the loop
        product_pi *= 1 / probability

    # Take the Nth root of the product
    perplexity = product_pi ** (1 / float(N))

    return perplexity


In [ ]:
# test your code

sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


<a name='4'></a>
## Part 4: Build an auto-complete system

In this section, you will combine the language models developed so far to implement an auto-complete system.


<a name='ex-11'></a>
Compute probabilities for all possible next words and suggest the most likely one.
- This function also take an optional argument `start_with`, which specifies the first few letters of the next words.

In [ ]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):
    # Length of previous words
    n = len(list(n_gram_counts.keys())[0])

    # From the words that the user already typed,
    # get the most recent 'n' words as the previous n-gram
    previous_n_gram = previous_tokens[-n:]

    # Estimate the probabilities that each word in the vocabulary
    # is the next word, given the previous n-gram,
    # the dictionary of n-gram counts, the dictionary of n plus 1 gram counts,
    # and the smoothing constant
    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)

    # Initialize suggested word to None. This will be set to the word with highest probability
    suggestion = None

    # Initialize the highest word probability to 0. This will be set to the highest probability of all words to be suggested
    max_prob = 0

    # For each word and its probability in the probabilities dictionary:
    for word, prob in probabilities.items():
        # If the optional start_with string is set and the beginning of word does not match with the letters in 'start_with', skip this word
        if start_with is not None and not word.startswith(start_with):
            continue

        # Check if this word's probability is greater than the current maximum probability
        if prob > max_prob:
            # If so, save this word as the best suggestion (so far)
            suggestion = word
            # Save the new maximum probability
            max_prob = prob

    return suggestion, max_prob


In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


### Get multiple suggestions

The function defined below loop over varioud n-gram models to get multiple suggestions.

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [ ]:
# test your code
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('cat', 0.1111111111111111),
 ('cat', 0.1111111111111111)]

### Suggest multiple words using n-grams of varying length

In [ ]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [ ]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('be', 0.027665685098338604),
 ('have', 0.00013487086115044844),
 ('have', 0.00013490725126475548),
 ('i', 6.746272684341901e-05)]

In [ ]:
previous_tokens = ["i", "want", "to", "go"]
tmp_suggest5 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest5)

The previous words are ['i', 'want', 'to', 'go'], the suggestions are:


[('to', 0.014051961029228078),
 ('to', 0.004697942168993581),
 ('to', 0.0009424436216762033),
 ('to', 0.0004044489383215369)]

In [ ]:
previous_tokens = ["hey", "how", "are"]
tmp_suggest6 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest6)

The previous words are ['hey', 'how', 'are'], the suggestions are:


[('you', 0.023426812585499317),
 ('you', 0.003559435862995299),
 ('you', 0.00013491635186184566),
 ('i', 6.746272684341901e-05)]

In [ ]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest7 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest7)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[("'re", 0.023973994311255586),
 ('?', 0.002888465830762161),
 ('?', 0.0016134453781512605),
 ('<e>', 0.00013491635186184566)]

In [ ]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('do', 0.009020723283218204),
 ('doing', 0.0016411737674785006),
 ('doing', 0.00047058823529411766),
 ('dvd', 6.745817593092283e-05)]